## Import

In [1]:
import pandas as pd, numpy as np
import re
from sklearn.preprocessing import MinMaxScaler

## Chargée les données

In [3]:
# Charger les données depuis le fichier CSV
data = pd.read_csv('../../Data/Raw/catastrophes_naturelles.csv')
data.head()

,Unnamed: 0,temperature,humidite,force_moyenne_du_vecteur_de_vent,force_du_vecteur_de_vent_max,pluie_intensite_max,date,quartier,sismicite,concentration_gaz,pluie_totale,catastrophe
0,0,5.7,75.6,3.8,4.3,0.0,2170-01-01,Zone 1,0.62,231.56,182.37,aucun
1,1,5.7,75.6,3.8,4.3,0.0,2170-01-01,Zone 2,0.94,248.20,903.20,['seisme']
2,2,5.7,75.6,3.8,4.3,0.0,2170-01-01,Zone 3,0.95,240.55,363.06,['seisme']
3,3,5.7,75.6,3.8,4.3,0.0,2170-01-01,Zone 4,0.39,159.10,473.93,aucun
4,4,5.7,75.6,3.8,4.3,0.0,2170-01-01,Zone 5,0.41,143.29,475.72,['innondation']


## Nettoyer les données et normaliser les donées numerique

In [4]:
# Nouvelle liste pour stocker les lignes corrigées
lignes_corrigees = []

for _, row in data.iterrows():
    # Supprimer les NaN pour compter les vraies valeurs
    non_vides = row.dropna().tolist()

    if len(non_vides) == 1:
        # Supprimer les guillemets
        contenu = non_vides[0].replace('"', '').strip()

        # Utiliser une expression régulière pour splitter seulement en dehors des crochets []
        valeurs = re.split(r',\s*(?![^\[]*\])', contenu)

        lignes_corrigees.append(valeurs)
    else:
        # Nettoyer chaque cellule (supprimer les guillemets)
        cleaned_row = [str(cell).replace('"', '') if isinstance(cell, str) else cell for cell in row.tolist()]
        lignes_corrigees.append(cleaned_row)

# Trouver le nombre maximum de colonnes dans les lignes corrigées
max_colonnes = max(len(ligne) for ligne in lignes_corrigees)

# Normaliser les lignes (compléter avec NaN pour que tout ait le même nombre de colonnes)
lignes_uniformes = [ligne + [np.nan] * (max_colonnes - len(ligne)) for ligne in lignes_corrigees]

# Créer un nouveau DataFrame propre
df = pd.DataFrame(lignes_uniformes)

In [5]:
import pandas as pd
import numpy as np
import re

# Renommer les colonnes (12 colonnes)
df.columns = [
    "id", "temperature", "humidite", "force_moyenne_du_vecteur_de_vent",
    "force_du_vecteur_de_vent_max", "pluie_intensite_max", "date", "quartier",
    "sismicite", "concentration_gaz", "pluie_totale", "catastrophe"
]

# Convertir la colonne date en datetime
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d', errors='coerce', utc=True)

# Nettoyer et normaliser la colonne 'catastrophe'
def clean_catastrophe(val):
    if isinstance(val, str):
        val = val.strip("[]").replace("'", "").strip()
    return val.lower()

df['catastrophe'] = df['catastrophe'].apply(clean_catastrophe)

def classify_catastrophe(cat):
    if 'seisme' not in cat and 'innondation' not in cat:
        return 'aucun'
    elif 'seisme' in cat and 'innondation' in cat:
        return 'seisme_et_inondation'
    elif 'seisme' in cat:
        return 'seisme'
    elif 'innondation' in cat:
        return 'innondation'
    else:
        return 'aucun'

df['catastrophe'] = df['catastrophe'].apply(classify_catastrophe)

# Extraire composants date
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day

# Extraire uniquement les chiffres après "Zone" dans la colonne quartier
def extract_zone_numbers(cell):
    if pd.isna(cell):
        return cell
    zones = cell.split(',')
    zones_numbers = []
    for zone in zones:
        m = re.search(r'Zone\s*(\d+)', zone)
        if m:
            zones_numbers.append(m.group(1))
        else:
            zones_numbers.append(zone.strip())
    return ','.join(zones_numbers)

df['quartier'] = df['quartier'].apply(extract_zone_numbers)
df['quartier'] = df['quartier'].str.split(',')

# Exploser la liste de quartiers pour avoir une ligne par quartier
df_exploded = df.explode('quartier')

# One-hot encoding des quartiers
quartier_dummies = pd.get_dummies(df_exploded['quartier'], prefix='quartier').astype(int)

# Joindre les colonnes one-hot au dataframe explosé
df_exploded = pd.concat([df_exploded, quartier_dummies], axis=1)

# Supprimer les colonne d'origine inutiles
df_final = df_exploded.drop(columns=['quartier', 'date', 'Year'])


In [6]:
# Colonnes à exclure de la conversion en numérique
colonnes_exclues = ['catastrophe']

# Convertir toutes les colonnes sauf les exclues en numérique
colonnes_a_convertir = [col for col in df_final.columns if col not in colonnes_exclues]

for col in colonnes_a_convertir:
    df_final[col] = pd.to_numeric(df_final[col], errors='coerce')

In [7]:
# Colonnes à exclure de la normalisation
colonnes_exclues = ['id', 'date', 'quartier']

# Sélection des colonnes numériques à normaliser
colonnes_a_normaliser = [
    col for col in df_final.select_dtypes(include=['float64', 'int64']).columns
    if col not in colonnes_exclues
]

# Appliquer MinMaxScaler
scaler = MinMaxScaler()
df_final[colonnes_a_normaliser] = scaler.fit_transform(df_final[colonnes_a_normaliser])

In [8]:
df_final.head()

,id,temperature,humidite,force_moyenne_du_vecteur_de_vent,force_du_vecteur_de_vent_max,pluie_intensite_max,sismicite,concentration_gaz,pluie_totale,catastrophe,Month,Day,quartier_1,quartier_2,quartier_3,quartier_4,quartier_5
0,0,0.465789,0.87237,0.086538,0.074653,0.0,0.457143,0.657992,0.182312,aucun,1,1,1,0,0,0,0
1,1,0.465789,0.87237,0.086538,0.074653,0.0,0.914286,0.741267,0.903308,seisme,1,1,0,1,0,0,0
2,2,0.465789,0.87237,0.086538,0.074653,0.0,0.928571,0.702983,0.363044,seisme,1,1,0,0,1,0,0
3,3,0.465789,0.87237,0.086538,0.074653,0.0,0.128571,0.295366,0.473939,aucun,1,1,0,0,0,1,0
4,4,0.465789,0.87237,0.086538,0.074653,0.0,0.157143,0.216245,0.475729,innondation,1,1,0,0,0,0,1


## Enregistrer les données nettoyées 

In [10]:
df_final.to_csv('../../Data/Preprocessed/catastrophes_naturelles_preprocessed.csv', index=False)